In [1]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

In [2]:
##
## Copyright (C) 2023, Collins Aerospace
## All rights reserved.
##
## This software may be modified and distributed under the terms
## of the 3-clause BSD license.  See the LICENSE file for details.
##


We are using Z3 to find DPSS configurations that result in extreme behavior.  The following functions provide a convenient interface to the Z3 facilities.


In [3]:
from z3 import *

In [4]:
def T(): return Not(False)
def F(): return Not(True)

def pyvalue(x):
    if is_true(x): return True
    if is_false(x): return False
    if is_int_value(x): return x.as_long()
    return x.numerator_as_long()/x.denominator_as_long()

## I have been seeing different results when replaying solveModel
## on the same input.  Perhaps by using a fresh context, those
## results will become more repeatable (?)

CTX = Context()

def prove(c):
    s = Solver(ctx=CTX)
    s.set("timeout", 20000)
    s.add(Not(c))
    return s.check() != sat

def solveModel(c):
    #context = new Context();
    #solver = context.mkSolver();
    #solver.add(context.mk...()); // Add some bool expr to the solver
    #Status status = solver.check();
    global CTX
    s = Solver(ctx=CTX)
    s.set("timeout", 20000)
    s.add(c)
    res = s.check()
    assert res != unknown
    if res != sat: return None
    m = s.model()
    return m
        
def modelValuesDict(m,ndict):
    ## ndict maps names to z3 expressions 
    res = {k:pyvalue(m.eval(e,model_completion=True)) for (k,e) in ndict.items()}
    return res

def modelValuesList(m,vlist):
    ## vlist is a list of z3 variables
    ndict = {str(v):v for v in vlist}
    return modelValuesDict(m,ndict)

def solve(c,vlist):
    m = solveModel(c)
    if m is None: return None
    return modelValuesList(m,vlist)

def optEval(e,*,m=None):
    if m is None: return e
    return pyvalue(m.eval(e))

x = Real('x',ctx=CTX)
xval = solve(And(100 < x,x < 1000),[x])['x']
print("A value for x that is greater than 100 :  x = {}".format(xval))

A value for x that is greater than 100 :  x = 101.0



We define an escort "Escort" data structure that aggregates the following DPSS coordination variables for a group of co-located UAVs.


| Variable | Description |
| :---- | :---- |
| L  | Number of UAVs to the left of the current Escort group |
| PL | Location of the Left Perimeter boundary |
| R  | Number of UAVs to the right of the current Escort group |
| PR | Location of the Right Perimeter boundary |
| E  | The number of UAVs in the Escort group |


In [5]:
class Escort():
    def __init__(self,*,name=None,L=None,PL=None,R=None,PR=None,E=1,time):
        global CTX
        if name is None: 
            assert not(L is None or PL is None or R is None or PR is None or E is None)
            pass
        if type(name) == str:
            L  = Real('{}.L'.format(name),ctx=CTX)  if L  is None else L
            PL = Real('{}.PL'.format(name),ctx=CTX) if PL is None else PL
            R  = Real('{}.R'.format(name),ctx=CTX)  if R  is None else R
            PR = Real('{}.PR'.format(name),ctx=CTX) if PR is None else PR
            pass
        if E is None:
            E = Real('{}.E'.format(name),ctx=CTX)
            pass
        self.name = name
        self.L  = L
        self.PL = PL
        self.R  = R
        self.PR = PR
        self.E  = E
        self.time = time
        return
    def update(self,*,L=None,PL=None,R=None,PR=None,E=None):
        L  = self.L  if L  is None else L
        PL = self.PL if PL is None else PL
        R  = self.R  if R  is None else R
        PR = self.PR if PR is None else PR        
        E  = self.E  if E  is None else E
        return self.new(name=self.name,L=L,PL=PL,R=R,PR=PR,E=E)
    def new(self,**kwargs):
        return self.__class__(**kwargs)
    pass


The `hyps()` predicate identifies some essential constraints on the coordination variables that characterize the Escort group's current state.

| Constraint | Description |
| :---- | :---- |
| `0 <= L`  | The number of UAVs to the left of the current Escort group is non-negative. |
| `0 <= R`  | The number of UAVs to the right of the current Escort group is non-negative. |
| `PL < PR` | The left perimeter boundary is left of the right perimeter boundary and the size of the perimeter is non-zero.  |
| `1 <= E`  | There is at least one UAV in the Escort group. |
| `Emin <= E` | There may be a lower bound on the number of UAVs in the Escort group |

In [6]:
def natp(x):
    return And(0 <= x,Implies(x != 0,x >= 1))

class Escort(Escort):
    def hyps(self,*,Emin=1,m=None):
        return optEval(And(natp(self.L), natp(self.R), self.PL < self.PR, 1 <= self.E, Emin <= self.E),m=m)
    pass


Using these coordination variables we can compute the Escorts current estimates for the following important parameters:


| Parameter | Computation | Description |
| :---- | :----: | :---- |
| `totalUAVS()` | `L + R + E` | Total number of UAVs on the perimeter |
| `perimeterLength()` | `PR - PL` | Length of the perimeter |
| `segmentLength()` | `perimeterLength()/totalUAVS()` | Length of the perimeter segment for each UAV on the perimeter |

In [7]:
class Escort(Escort):
    def totalUAVS(self,m=None):
        return optEval(self.L + self.R + self.E,m=m)
    def perimeterLength(self,m=None):
        return optEval(self.PR - self.PL,m=m)
    def segmentLength(self,m=None):
        return optEval(self.perimeterLength()/self.totalUAVS(),m=m)
    def density(self,*,m=None):
        res = self.totalUAVS()/self.perimeterLength()
        return optEval(res,m=m)
    pass


An Escort group consists of a number of co-located UAVs.  Such a group must consist of adjacent UAVs.  Thus, the segments survailed by the group will be adjacent and sequential. Using the Escort parameters, we can compute the leftmost and rightmost boundaries of the Escort group's segments.  We can also define the segment boundaries for each UAV within the Escort group.

| Parameter | Computation |
| :---- | :---- |
| `leftmostSegmentBoundary()`   | `PL + L*segmentLength()` |
| `rightmostSegmentBoundary()`  | `PL + (L+E)*segmentLength()` |
| `leftSegmentBoundary(uid)`        | `PL + (L + uid)*segmentLength()` |
| `rightEscortSegmentBoundary(uid)` | `PL + (L+uid+1)*segmentLength()` |



In [8]:
class Escort(Escort):
    def leftSegmentBoundary(self,uid):
        return self.PL + (self.L + uid)*self.segmentLength()
    def rightSegmentBoundary(self,uid):
        return self.PL + (self.L + uid + 1)*self.segmentLength()
    def leftmostSegmentBoundary(self,*,m=None):
        return optEval(self.PL + self.L*self.segmentLength(),m=m)
    def leftmostMidpoint(self,*,m=None):
        return optEval((self.leftSegmentBoundary(0) + self.rightSegmentBoundary(0))/2.0,m=m)
    def rightmostSegmentBoundary(self,*,m=None):
        return optEval(self.PL + (self.L + self.E)*self.segmentLength(),m=m)
    def rightmostMidpoint(self,*,m=None):
        uid = self.E - 1
        return optEval((self.leftSegmentBoundary(uid) + self.rightSegmentBoundary(uid))/2.0,m=m)
    def escortMidpoint(self,*,m=None):
        return optEval((self.leftmostSegmentBoundary() + self.rightmostSegmentBoundary())/2,m=m)
    pass


Here we illustrate two Escorts .. one that violates the hypotheses and another that satisfies them.

In [9]:
print()
uav = Escort(name="Bad",L=-1,PL=0,R=2,PR=-3,time=0)
print("{}  Escort: {}".format(uav.name,simplify(uav.hyps())))

uav = Escort(name="Good",L=0,PL=0,R=1,PR=100,time=0)
print("{} Escort: {}".format(uav.name,simplify(uav.hyps())))


Bad  Escort: False
Good Escort: True


At what point does the first member of the escort leave the group? When it reaches its most extreme segment boundary in the direction of motion.  When moving to the right, it is the right sement boundary of the leftmost UAV in the Escort.  When moving to the left it is the left segment boundary of the rightmost UAV in the Escort.

In [10]:
class Escort(Escort):
    def splitLocation_MovingRight(self,*,m=None):
        return optEval(self.rightSegmentBoundary(0),m=m)
    def splitLocation_MovingLeft(self,*,m=None):
        return optEval(self.leftSegmentBoundary(self.E-1),m=m)
    pass


To force Z3 to produce values that are "obviously" bigger than or less than a given value, we add an 'epsilon' parameter.

In [11]:
class Escort(Escort):
    epsilon = 0.001
    def lt(self,x,y):
        return x + Escort.epsilon <= y
    pass


We are interested in bounding the split location of the Escort.  The `splitBound()` functions provide a means of expressing upper and lower bounds on the split location.  The `simpleBound()` functions provide a means of expressing just lower bounds on the split location. 

In [12]:
class Escort(Escort):
    def splitBound_MovingRight(self,*,lowerBound,upperBound=None):
        if upperBound is None: return self.lt(lowerBound,self.splitLocation_MovingRight())
        return And(self.lt(lowerBound,self.splitLocation_MovingRight()),
                   self.lt(self.splitLocation_MovingRight(),upperBound))
    def splitBound_MovingLeft(self,*,lowerBound=None,upperBound):
        if lowerBound is None: return self.lt(self.splitLocation_MovingLeft(),upperBound)
        return And(self.lt(lowerBound,self.splitLocation_MovingLeft()),
                   self.lt(self.splitLocation_MovingLeft(),upperBound))
    pass


To help interpret Z3 solutions we need the identity of the various "free" coordination variables from the Escort.

In [13]:
class Escort(Escort):
    def variables(self):
        return set([self.L,self.PL,self.R,self.PR,self.E])
    pass


The counterexample needs a "guard" UAV that will protect the remaining UAVs from the new information by deflecting their direction of travel away from the carrier of the new information.

PROPERTY: If the first UAV is not a sufficiently capable guard, no subsequent UAV from the ensemble will be.

ie: Every subsequent UAV will be a "worse" guard.


In [14]:
class Escort(Escort):
    def getUAV(self,*,i):
        L  = self.L+i
        PL = self.PL
        R  = self.R+self.E-i-1
        PR = self.PR
        E  = 1
        time = self.time
        return self.new(name='anonymous UAV',L=L,PL=PL,R=R,PR=PR,E=E,time=time)
    def remove_Rightmost_UAV(self,*,time):
        L  = self.L 
        PL = self.PL
        R  = self.R+1
        PR = self.PR
        E  = self.E-1
        return self.new(name=self.name,L=L,PL=PL,R=R,PR=PR,E=E,time=time)
    def remove_Leftmost_UAV(self,*,time):
        L  = self.L+1
        PL = self.PL
        R  = self.R
        PR = self.PR
        E  = self.E-1
        return self.new(name=self.name,L=L,PL=PL,R=R,PR=PR,E=E,time=time)
    pass


Using the primitive `getUAV()` and `remove_UAV()` primitives we can define `extract_UAV()` operations 
that extract a UAV from an Escort and return the individual UAV and the remaining Escort.

In [15]:
class Escort(Escort):
    def extract_Rightmost_UAV(self,*,time):
        return (self.remove_Rightmost_UAV(time=time),self.getUAV(i=self.E-1))
    def extract_Leftmost_UAV(self,*,time):
        return (self.getUAV(i=0),self.remove_Leftmost_UAV(time=time))
    pass


We now introduce functions for adding (ostensibly) individual UAVs to Escort groups.

| Parameter | Computation |
| :---- | :---- |
| `addUAV_fromRight(Right)`  | Adds a UAV from the right (Right) to the current Escort group. |
| `addUAV_fromLeft(Right)`    | Adds the current UAV to the Escort group to the right (Right)  |



In [16]:
class Escort(Escort):
    def addUAV_fromRight(self,*,Right,time):
        Left = self
        L  = Left.L 
        PL = Left.PL
        R  = Right.R
        PR = Right.PR
        E  = Left.E + Right.E
        return self.new(name=Left.name,L=L,PL=PL,R=R,PR=PR,E=E,time=time)
    def addUAV_fromLeft(self,*,Right,time):
        Left = self
        L  = Left.L
        PL = Left.PL
        R  = Right.R
        PR = Right.PR
        E  = Left.E+Right.E
        return self.new(name=Right.name,L=L,PL=PL,R=R,PR=PR,E=E,time=time)
    def impactLeftBoundary(self,*,PL,time):
        L  = 0
        PL = PL
        R  = self.R
        PR = self.PR
        E  = self.E
        return self.new(name=self.name,L=L,PL=PL,R=R,PR=PR,E=E,time=time)
    def impactRightBoundary(self,*,PR,time):
        L  = self.L
        PL = self.PL
        R  = 0
        PR = PR
        E  = self.E
        return self.new(name=self.name,L=L,PL=PL,R=R,PR=PR,E=E,time=time)
    pass


In [17]:
class Escort(Escort):
    def printState(self,m):
        L  = pyvalue(m.eval(self.L))
        PL = pyvalue(m.eval(self.PL))
        R  = pyvalue(m.eval(self.R))
        PR = pyvalue(m.eval(self.PR))
        E  = pyvalue(m.eval(self.E))
        print("{}_{}:[\n PL=RealVal({},ctx=CTX),\n L =RealVal({},ctx=CTX),\n R =RealVal({},ctx=CTX),\n PR=RealVal({},ctx=CTX),\n E =RealVal({},ctx=CTX)\n]".format(self.name,self.time,PL,L,R,PR,E))
        return
    def printHyps(self,m):
        print("{}_{}.hyp() = {}".format(self.name,self.time,m.eval(self.hyps())))
        return
    def isEqual(self,other):
        return And([self.L == other.L,
                    self.PL == other.PL,
                    self.R == other.R,
                    self.PR == other.PR,
                    self.E == other.E
                   ])
    pass


In [18]:
def orderingHyps(x,*args):
    args = [x] + list(args)
    a = args.pop()
    PL = a.PL
    PR = a.PR
    for a in args:
        nL = a.PL
        nR = a.PR
        PL = If(PL <= nL,nL,PL)
        PR = If(nR <= PR,nR,PR)
    return PL < PR


In [19]:
def ensembleHyps(x,y,*args,movingLeft=False,movingRight=False,m=None):
    assert movingLeft != movingRight
    args = [x,y] + list(args)
    res = [orderingHyps(*args)]
    ## We allow the "trailing" escort to have just one
    ex = args.pop(0) if movingRight else args.pop()
    res += [ex.hyps()]
    for ei in args:
        res += [ei.hyps(Emin=2)]
    return optEval(And(res),m=m)

E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

hyp_0 = And(E0_0.hyps(),E1_0.hyps(Emin=2),E2_0.hyps(Emin=2),orderingHyps(E0_0,E1_0,E2_0))

hyp_x = ensembleHyps(E0_0,E1_0,E2_0,movingRight=True)

ok = prove(hyp_0 == hyp_x)
print("\nIs ensembleHyps() reasonable? {}".format("Yes!" if ok else "NO!"))
assert ok



Is ensembleHyps() reasonable? Yes!


In [20]:
def totalUAVS(*args,m=None):
    total = 0
    for a in args: total += a.E
    return optEval(total,m=m)
    
def perimeterDensity(*args,PL=None,PR=None,m=None):
    assert PL is not None and PR is not None
    total = totalUAVS(*args)
    return optEval(total/(PR - PL),m=m)
    

In [21]:
def absorbBlockersFromRight(x,y,*args,time=None):
    assert time is not None
    args = [x,y] + list(args)
    e0 = args.pop(0)
    (uavs,args) = zip(*[ei.extract_Leftmost_UAV(time=time) for ei in args])
    args = [e0] + list(args)
    eN = args.pop()
    args = [ei.addUAV_fromRight(Right=uavi,time=time) for (ei,uavi) in zip(args,uavs)]
    args = args + [eN]
    return tuple(args)

E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

(UAV1,E1_1) = E1_0.extract_Leftmost_UAV(time=1)
(UAV2,E2_1) = E2_0.extract_Leftmost_UAV(time=1)

E0_1 = E0_0.addUAV_fromRight(Right=UAV1,time=1)
E1_1 = E1_1.addUAV_fromRight(Right=UAV2,time=1)
E2_1 = E2_1

(E0_x,E1_x,E2_x) = absorbBlockersFromRight(E0_0,E1_0,E2_0,time=1)

ok = prove(And(E0_x.isEqual(E0_1),E1_x.isEqual(E1_1),E2_x.isEqual(E2_1)))
print("\nIs absorbBlockersFromRight() reasonable? {}".format("Yes!" if ok else "NO!"))
assert ok



Is absorbBlockersFromRight() reasonable? Yes!


In [22]:
def conferBlockersToRight(x,y,*args,time=None):
    assert time is not None
    args = [x,y] + list(args)
    eN = args.pop()
    (args,uavs) = zip(*[ei.extract_Rightmost_UAV(time=time) for ei in args])
    args = list(args) + [eN]
    e0 = args.pop(0)
    args = [uavi.addUAV_fromLeft(Right=ei,time=time) for (uavi,ei) in zip(uavs,args)]
    args = [e0] + args
    return tuple(args)

E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

(E0_1,UAV0) = E0_0.extract_Rightmost_UAV(time=1)
(E1_1,UAV1) = E1_0.extract_Rightmost_UAV(time=1)

E0_1 = E0_1
E1_1 = UAV0.addUAV_fromLeft(Right=E1_1,time=1)
E2_1 = UAV1.addUAV_fromLeft(Right=E2_0,time=1)

(E0_x,E1_x,E2_x) = conferBlockersToRight(E0_0,E1_0,E2_0,time=1)

ok = prove(And(E0_x.isEqual(E0_1),E1_x.isEqual(E1_1),E2_x.isEqual(E2_1)))
print("\nIs conferBlockersToRight() reasonable? {}".format("Yes!" if ok else "NO!"))
assert ok



Is conferBlockersToRight() reasonable? Yes!


In [23]:
E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

hypR_0 = ensembleHyps(E0_0,E1_0,E2_0,movingRight=True)
ok = solveModel(hypR_0) is not None
print("\nAre ensembleHyps(movingRight=True) satisfiable? {}".format("Yes!" if ok else "NO!"))
assert ok

(E0_1,E1_1,E2_1) = absorbBlockersFromRight(E0_0,E1_0,E2_0,time=1)
hypL_1 = ensembleHyps(E0_1,E1_1,E2_1,movingLeft=True)

ok = prove(Implies(hypR_0,hypL_1))
print("\nAre ensembleHyps() invariant over left turns? {}".format("Yes!" if ok else "NO!"))
assert ok



Are ensembleHyps(movingRight=True) satisfiable? Yes!

Are ensembleHyps() invariant over left turns? Yes!


In [24]:
E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

hypL_0 = ensembleHyps(E0_0,E1_0,E2_0,movingLeft=True)
ok = solveModel(hypL_0) is not None
print("\nAre ensembleHyps(movingLeft=True)  satisfiable? {}".format("Yes!" if ok else "NO!"))
assert ok

(E0_1,E1_1,E2_1) = conferBlockersToRight(E0_0,E1_0,E2_0,time=1)
hypR_1 = ensembleHyps(E0_1,E1_1,E2_1,movingRight=True)

ok = prove(Implies(hypL_0,hypR_1))
print("\nAre ensembleHyps() invariant over right turns? {}".format("Yes!" if ok else "NO!"))
assert ok



Are ensembleHyps(movingLeft=True)  satisfiable? Yes!

Are ensembleHyps() invariant over right turns? Yes!


In [25]:
def ensembleSplitBounds_MovingRight(e0,*args,lowerBound=None,upperBound=None,m=None):
    assert lowerBound is not None
    args = list(args)
    res = []
    while args:
        e1 = args.pop(0)
        splitLocation = e1.splitLocation_MovingRight()
        res.append(e0.splitBound_MovingRight(lowerBound=lowerBound,upperBound=splitLocation))
        lowerBound = e0.splitLocation_MovingRight()
        e0 = e1
        continue
    res.append(e0.splitBound_MovingRight(lowerBound=lowerBound,upperBound=upperBound))
    return optEval(And(res),m=m)

E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

s0 = E0_0.splitLocation_MovingRight()
s1 = E1_0.splitLocation_MovingRight()
s2 = E2_0.splitLocation_MovingRight()

b0 = E0_0.splitBound_MovingRight(lowerBound=98,upperBound=s1)
b1 = E1_0.splitBound_MovingRight(lowerBound=s0,upperBound=s2)
b2 = E2_0.splitBound_MovingRight(lowerBound=s1,upperBound=100)

bounds_0 = And(b0,b1,b2)

bounds_x = ensembleSplitBounds_MovingRight(E0_0,E1_0,E2_0,lowerBound=98,upperBound=100)

ok = prove(bounds_0 == bounds_x)
print("\nIs ensembleSplitBounds_MovingRight() reasonable? {}".format("Yes!" if ok else "NO!"))
assert ok

hyp_0 = ensembleHyps(E0_0,E1_0,E2_0,movingRight=True)

m = solveModel(And(hyp_0,bounds_x))
print("\nCan we find a state that staisfies these bounds? {}\n".format("Yes!" if m else "NO!"))
assert m

print("E0 split = {}".format(pyvalue(m.eval(E0_0.splitLocation_MovingRight()))))
print("E1 split = {}".format(pyvalue(m.eval(E1_0.splitLocation_MovingRight()))))
print("E2 split = {}".format(pyvalue(m.eval(E2_0.splitLocation_MovingRight()))))



Is ensembleSplitBounds_MovingRight() reasonable? Yes!

Can we find a state that staisfies these bounds? Yes!

E0 split = 98.1311919504644
E1 split = 98.24093264248705
E2 split = 98.4922077922078


In [26]:
def ensembleSplitBounds_MovingLeft(e0,*args,lowerBound=None,upperBound=None,m=None):
    assert upperBound is not None
    args = [e0] + list(args)
    e1 = args.pop()
    res = []
    while args:
        e0 = args.pop()
        splitLocation = e0.splitLocation_MovingLeft()
        res.append(e1.splitBound_MovingLeft(lowerBound=splitLocation,upperBound=upperBound))
        upperBound = e1.splitLocation_MovingLeft()
        e1 = e0
        continue
    res.append(e1.splitBound_MovingLeft(lowerBound=lowerBound,upperBound=upperBound))
    return optEval(And(res),m=m)

E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

s0 = E0_0.splitLocation_MovingLeft()
s1 = E1_0.splitLocation_MovingLeft()
s2 = E2_0.splitLocation_MovingLeft()

b0 = E0_0.splitBound_MovingLeft(lowerBound=0,upperBound=s1)
b1 = E1_0.splitBound_MovingLeft(lowerBound=s0,upperBound=s2)
b2 = E2_0.splitBound_MovingLeft(lowerBound=s1,upperBound=2)

bounds_0 = And(b0,b1,b2)

bounds_x = ensembleSplitBounds_MovingLeft(E0_0,E1_0,E2_0,lowerBound=0,upperBound=2)

ok = prove(bounds_0 == bounds_x)
print("\nIs ensembleSplitBoundsMovingRight() reasonable? {}".format("Yes!" if ok else "NO!"))
assert ok

hyp_0 = ensembleHyps(E0_0,E1_0,E2_0,movingLeft=True)

m = solveModel(And(hyp_0,bounds_x))
print("\nCan we find a state that satisfies these bounds? {}\n".format("Yes!" if m else "NO!"))
assert m

print("E0 split = {}".format(pyvalue(m.eval(E0_0.splitLocation_MovingLeft()))))
print("E1 split = {}".format(pyvalue(m.eval(E1_0.splitLocation_MovingLeft()))))
print("E2 split = {}".format(pyvalue(m.eval(E2_0.splitLocation_MovingLeft()))))



Is ensembleSplitBoundsMovingRight() reasonable? Yes!

Can we find a state that satisfies these bounds? Yes!

E0 split = 0.7857142857142857
E1 split = 0.9285714285714286
E2 split = 1.0625


This is a simple experiment to check a hypothesis regarding extreme bounds.

In [27]:
CTX = Context()
set_option('smt.random_seed', 2343)

E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)
E3_0 = Escort(name="E3",E=None,time=0)

h1 = E1_0.hyps(Emin=2)
h2 = E2_0.hyps(Emin=2)
h3 = E3_0.hyps(Emin=2)

hyps = And(h1,h2,h3,orderingHyps(E1_0,E2_0,E3_0))

##  | lowerBound          upperBound |
##       \        \         \
##      E1_0     E2_0      E3_0
##      /  \      / \       / \
##   UAV1  E1  UAV2  E2  UAV3  E3
##    /     \  /      \  /      \
##         E1_1      E2_1
##          /         /

bx_0 = ensembleSplitBounds_MovingRight(E1_0,E2_0,E3_0,lowerBound=90,upperBound=100)

(UAV1_1,E1_1) = E1_0.extract_Leftmost_UAV(time=1)
(UAV2_1,E2_1) = E2_0.extract_Leftmost_UAV(time=1)
(UAV3_1,E3_1) = E3_0.extract_Leftmost_UAV(time=1)
E3_1 = E3_1.impactRightBoundary(PR=100,time=1)

E1_1 = E1_1.addUAV_fromRight(Right=UAV2_1,time=1)
E2_1 = E2_1.addUAV_fromRight(Right=UAV3_1,time=1)

assert prove(totalUAVS(UAV1_1,E1_1,E2_1,E3_1) == totalUAVS(E1_0,E2_0,E3_0))

(UAV2_2,E2_2) = E2_1.extract_Rightmost_UAV(time=2)
E3_2 = UAV2_2.addUAV_fromLeft(Right=E3_1,time=2)

##  | lowerBound          upperBound |
##          /          /
##       E1_1       E2_1
##       /  \       /  \
##     E1  UAV1   E2  UAV2  E3_1
##    /      \   /      \   /
##           E2_2       E3_2
##             \          \

## We start by focusing on a single solution ..

bx_1 = ensembleSplitBounds_MovingLeft(E2_1,lowerBound=0,upperBound=10)

bx_2 = ensembleSplitBounds_MovingRight(E3_2,lowerBound=90,upperBound=100)

#dx = And(E3_0.density() < perimeterDensity(E2_0,E3_0,PL=0,PR=100))

#print(prove(Implies(And(hyps,bx_0,bx_1),E0_0.density() <= 2*E1_0.density())))

c = And(hyps,bx_0,bx_1,bx_2)
m = solveModel(c)

if m is None: 
    print("Fail")

print()
print("E1_0.density      = {}".format(E1_0.density(m=m)))
print("E2_0.density      = {}".format(E2_0.density(m=m)))
print("E3_0.density      = {}".format(E3_0.density(m=m)))
print("Perimeter density = {}".format(perimeterDensity(E2_0,E3_0,PL=0,PR=100,m=m)))
print()




E1_0.density      = 2.9375
E2_0.density      = 0.09088221381267739
E3_0.density      = 46.001953125
Perimeter density = 0.04009765625



In [28]:
CTX = Context()
#set_option('smt.random_seed', 137)
set_option('smt.random_seed', 133577)

E0_0 = Escort(name="E0",E=None,time=0)
E1_0 = Escort(PL=RealVal(-704.654296875,ctx=CTX),
              L =RealVal(2.0,ctx=CTX),
              R =RealVal(0.0,ctx=CTX),
              PR=RealVal(354.896484375,ctx=CTX),
              E =RealVal(2.000030517578125,ctx=CTX),
              name="E1",time=0)
#E1_0 = Escort(PL=RealVal(-2619.0,ctx=CTX),
#              L =RealVal(26.0,ctx=CTX),
#              R =RealVal(2.0,ctx=CTX),
#              PR=RealVal(480.0,ctx=CTX),
#              E =RealVal(3.0,ctx=CTX),
#              name="E1",time=0)
#
#E1_0 = Escort(name="E1",E=None,time=0)
E2_0 = Escort(name="E2",E=None,time=0)

h0 = E0_0.hyps(Emin=2)
h1 = E1_0.hyps(Emin=2)
h2 = E2_0.hyps(Emin=2)
oh = orderingHyps(E0_0,E1_0,E2_0)


hyps = And(h0,h1,h2,oh)

##  | lowerBound          upperBound |
##       \        \         \
##      E0_0     E1_0      E2_0
##         \      / \       / \
##         E0  UAV1  E1  UAV2  E2
##          \  /      \  /      \
##         E0_1      E1_1      E2_1 |
##          /         /         /

bx_0 = ensembleSplitBounds_MovingRight(E1_0,E2_0,lowerBound=80,upperBound=100)

(UAV1_0,E1_1) = E1_0.extract_Leftmost_UAV(time=1)
(UAV2_0,E2_1) = E2_0.extract_Leftmost_UAV(time=1)

E0_1 = E0_0.addUAV_fromRight(Right=UAV1_0,time=1)
E1_1 = E1_1.addUAV_fromRight(Right=UAV2_0,time=1)
E2_1 = E2_1.impactRightBoundary(PR=100,time=1)

## | lowerBound          upperBound |
##            /          /
##         E0_1       E1_1
##         /  \       /  \
##       E0  UAV0   E1  UAV1  E2_1
##      /      \   /      \  /
## | E0_2      E1_2       E2_2
##     \         \          \
##


bx_1 = ensembleSplitBounds_MovingLeft(E0_1,E1_1,lowerBound=0,upperBound=10)

(E0_2,UAV0_1) = E0_1.extract_Rightmost_UAV(time=2)
(E1_2,UAV1_1) = E1_1.extract_Rightmost_UAV(time=2)

E0_2 = E0_2.impactLeftBoundary(PL=0,time=2)
E1_2 = UAV0_1.addUAV_fromLeft(Right=E1_2,time=2)
E2_2 = UAV1_1.addUAV_fromLeft(Right=E2_1,time=2)

bx_2 = ensembleSplitBounds_MovingRight(E0_2,E1_2,lowerBound=80,upperBound=100.0)

##
## Does that permit a full solution?
##

c = And(hyps,bx_0,bx_1,bx_2)

##
## This reflects our attempted proof above.
## It does not, however, provide us with a useful
## constraint.

print("Hard upper bound on E2? {}".format(prove(Implies(c,E2_0.density() < 2*E1_0.density()))))

## So, probably given the low
## density of E1 (relative to the size of the perimeter) we 
## find that E2 has a hard lower bound.
##
## This appears consistent with the zig-zag pattern.
##
#print("Hard lower bound on E2? {}".format(prove(Implies(c,E1_0.density() < E2_0.density()))))

print("Constant upper bound on E1? {}".format(prove(Implies(c,E1_0.density() < 1.0/100.0))))
print("Constant upper bound on E2? {}".format(prove(Implies(c,E2_0.density() < 1.0/100.0))))

#print("Impossible? {}".format(prove(Implies(And(hyps,bx_0,bx_1),Not(bx_2)))))

m = solveModel(c)

#p = Implies(c,And(E1_0.density() < E0_0.density(),E1_0.density() < E2_0.density()))

#print("proof : {}".format(prove(p)))

## Interesting .. so Z3 finds a counterexample when seeded with a partial solution.
## If not, Z3 proves it is impossible.
if m is None:
    print()
    print("UNSAT")
if m is not None:
    print()
    E1_0.printState(m=m)
    print()
    print("E0_0.total = {}".format(E0_0.totalUAVS(m=m)))
    print("E1_0.total = {}".format(E1_0.totalUAVS(m=m)))
    print("E2_0.total = {}".format(E2_0.totalUAVS(m=m)))
    print()
    print("E0_0.midpoint = {}".format(E0_0.escortMidpoint(m=m)))
    print("E1_0.midpoint = {}".format(E1_0.escortMidpoint(m=m)))
    print("E2_0.midpoint = {}".format(E2_0.escortMidpoint(m=m)))
    print()
    print()
    print("E0_1.midpoint = {}".format(E0_1.escortMidpoint(m=m)))
    print("E1_1.midpoint = {}".format(E1_1.escortMidpoint(m=m)))
    print("E2_1.midpoint = {}".format(E2_1.escortMidpoint(m=m)))
    print()
    print("E0_2.midpoint = {}".format(E0_2.escortMidpoint(m=m)))
    print("E1_2.midpoint = {}".format(E1_2.escortMidpoint(m=m)))
    print("E2_2.midpoint = {}".format(E2_2.escortMidpoint(m=m)))
    print()
    print("Perimeter density = {}".format(perimeterDensity(E0_0,E1_0,E2_0,PL=0,PR=100,m=m)))
    print()
    print("E0_0.density = {}".format(E0_0.density(m=m)))
    print("E1_0.density = {}".format(E1_0.density(m=m)))
    print("E2_0.density = {}".format(E2_0.density(m=m)))
    print()
    print("E0_1.density = {}".format(E0_1.density(m=m)))
    print("E1_1.density = {}".format(E1_1.density(m=m)))
    print("E2_1.density = {}".format(E2_1.density(m=m)))
    print()
    print("E0_2.density = {}".format(E0_2.density(m=m)))
    print("E1_2.density = {}".format(E1_2.density(m=m)))
    print("E2_2.density = {}".format(E2_2.density(m=m)))
    print()
    print("E0_0.split = {}".format(E0_0.splitLocation_MovingRight(m=m)))
    print("E1_0.split = {}".format(E1_0.splitLocation_MovingRight(m=m)))
    print("E2_0.split = {}".format(E2_0.splitLocation_MovingRight(m=m)))
    print()
    print("E0_1.split = {}".format(E0_1.splitLocation_MovingLeft(m=m)))
    print("E1_1.split = {}".format(E1_1.splitLocation_MovingLeft(m=m)))
    print("E2_1.split = {}".format(E2_1.splitLocation_MovingLeft(m=m)))
    print()
    print("E0_2.split = {}".format(E0_2.splitLocation_MovingRight(m=m)))
    print("E1_2.split = {}".format(E1_2.splitLocation_MovingRight(m=m)))
    print("E2_2.split = {}".format(E2_2.splitLocation_MovingRight(m=m)))
 


Hard upper bound on E2? True
Constant upper bound on E1? True
Constant upper bound on E2? True

E1_0:[
 PL=RealVal(-704.654296875,ctx=CTX),
 L =RealVal(2.0,ctx=CTX),
 R =RealVal(0.0,ctx=CTX),
 PR=RealVal(354.896484375,ctx=CTX),
 E =RealVal(2.000030517578125,ctx=CTX)
]

E0_0.total = 6.25
E1_0.total = 4.000030517578125
E2_0.total = 13.0

E0_0.midpoint = -362.25
E1_0.midpoint = 90.00676814518437
E2_0.midpoint = 749.5


E0_1.midpoint = -98.24309451614006
E1_1.midpoint = 9.654787227507192
E2_1.midpoint = -109.0909090909091

E0_2.midpoint = 93.94251247621442
E1_2.midpoint = 97.13136865346068
E2_2.midpoint = -178.53352582837425

Perimeter density = 0.13250030517578126

E0_0.density = 0.008638562543192813
E1_0.density = 0.003775213598407344
E2_0.density = 0.005308289097590854

E0_1.density = 0.005792984413327419
E1_1.density = 0.0055998381482364554
E2_1.density = 0.019130434782608695

E0_2.density = 0.011975408900042376
E1_2.density = 0.006393636218524138
E2_2.density = 0.016156044363481017

E

In [29]:
print("Done.")

Done.


In [30]:
CTX = Context()
set_option('smt.random_seed', 137)

UAV0_0 = Escort(name="UAV0",E=RealVal(1,ctx=CTX),time=0)
EE12_0 = Escort(name="EE12",E=RealVal(2,ctx=CTX),time=0)
UAV3_0 = Escort(name="UAV3",E=RealVal(1,ctx=CTX),time=0)

S12_0 = EE12_0.splitLocation_MovingRight()

(UAV1_1,UAV2_1) = EE12_0.extract_Leftmost_UAV(time=1)

EE01_1 = UAV0_0.addUAV_fromRight(Right=UAV1_1,time=1)
EE23_1 = UAV2_1.addUAV_fromRight(Right=UAV3_0,time=1)

S01_1 = EE01_1.splitLocation_MovingLeft()
S23_1 = EE23_1.splitLocation_MovingLeft()

(UAV0_2,UAV1_2) = EE01_1.extract_Rightmost_UAV(time=2)
(UAV2_2,UAV3_2) = EE23_1.extract_Leftmost_UAV(time=2)

EE12_2 = UAV1_2.addUAV_fromLeft(Right=UAV2_2,time=2)

L12_2 = EE12_2.leftmostSegmentBoundary()
S12_2 = EE12_2.splitLocation_MovingRight()

h0 = UAV0_0.hyps()
h1 = EE12_0.hyps()
h2 = UAV3_0.hyps()
h3 = EE01_1.hyps()
h4 = EE23_1.hyps()
h5 = EE12_2.hyps()

## These hyps capture what we consider to be fundamental and 
## reasonable constraints on the configuration.

hyps = And(h0,h1,h2,h3,h4,h5)

In [31]:
##
##
## Ideal Impossibility Theorem
##
##

## Here are our minimal hypotheses .. that the secondary
## split points are to the left of the primary split point.
HA = And(hyps,S01_1 < S12_0,S23_1 < S12_0)

## We want to conclude that the final escort split cannot
## be to the right of both secondary split points.
C0 = Not(And(S01_1 < S12_2,S23_1 < S12_2))

## The ideal impossibility theorem mirrors our scalar impossibility
## theorem.  Sadly, however the theorem as stated is not true.

## Here we demonstrate a counterexample to this formulation of
## the simplest impossibility proposition.  An actual proof will
## requires at least one additional constraint.

m = solveModel(And(HA,Not(C0)))

UAV0_0.printState(m=m)
EE12_0.printState(m=m)
UAV3_0.printState(m=m)

print("\nEE12:")
print(EE12_0.density(m=m),EE12_0.segmentLength(m=m))
print("M1  = {}".format(EE12_0.leftmostMidpoint(m=m)))
print("S12 = {}".format(EE12_0.splitLocation_MovingRight(m=m)))
print("M2  = {}".format(EE12_0.rightmostMidpoint(m=m)))

print("\nEE01:")
print(EE01_1.density(m=m),EE01_1.segmentLength(m=m))
print("M0  = {}".format(EE01_1.leftmostMidpoint(m=m)))
print("S01 = {}".format(EE01_1.splitLocation_MovingLeft(m=m)))
print("M1  = {}".format(EE01_1.rightmostMidpoint(m=m)))

print("\nEE23:")
print(EE23_1.density(m=m),EE23_1.segmentLength(m=m))
print("M2  = {}".format(EE23_1.leftmostMidpoint(m=m)))
print("S23 = {}".format(EE23_1.splitLocation_MovingLeft(m=m)))
print("M3  = {}".format(EE23_1.rightmostMidpoint(m=m)))

print("\nEE12:")
print(EE12_2.density(m=m),EE12_2.segmentLength(m=m))
print("L12_2  = {}".format(EE12_2.leftmostSegmentBoundary(m=m)))
print("M1     = {}".format(EE12_2.leftmostMidpoint(m=m)))
print("S12    = {}".format(EE12_2.splitLocation_MovingRight(m=m)))
print("M2     = {}".format(EE12_2.rightmostMidpoint(m=m)))
print()
print("PR = {}".format(pyvalue(m.eval(EE12_2.PR))))

UAV0_0:[
 PL=RealVal(-0.25,ctx=CTX),
 L =RealVal(2.0,ctx=CTX),
 R =RealVal(2.0,ctx=CTX),
 PR=RealVal(0.75,ctx=CTX),
 E =RealVal(1.0,ctx=CTX)
]
EE12_0:[
 PL=RealVal(-0.5,ctx=CTX),
 L =RealVal(2.0,ctx=CTX),
 R =RealVal(2.0,ctx=CTX),
 PR=RealVal(2.0,ctx=CTX),
 E =RealVal(2.0,ctx=CTX)
]
UAV3_0:[
 PL=RealVal(0.9375,ctx=CTX),
 L =RealVal(2.0,ctx=CTX),
 R =RealVal(3.0,ctx=CTX),
 PR=RealVal(1.9375,ctx=CTX),
 E =RealVal(1.0,ctx=CTX)
]

EE12:
2.4 0.4166666666666667
M1  = 0.5416666666666666
S12 = 0.75
M2  = 0.9583333333333334

EE01:
3.111111111111111 0.32142857142857145
M0  = 0.5535714285714286
S01 = 0.7142857142857143
M1  = 0.875

EE23:
3.282051282051282 0.3046875
M2  = 0.56640625
S23 = 0.71875
M3  = 0.87109375

EE12:
4.114285714285714 0.24305555555555555
L12_2  = 0.4791666666666667
M1     = 0.6006944444444444
S12    = 0.7222222222222222
M2     = 0.84375

PR = 1.9375


In [32]:

## OK .. what can we do to articulate a reasonable theorem?

## The final escort group is right coordinated.  So it knows
## the actual right perimeter and it knows that there are no
## additional UAVs to the right.  So in the following theorems,
## we are using L12_2 as a reference point for what a "reasonable"
## counterexample would have to look like.


##
##    ----+-----+-----|
##      L12_2 S12_2  RP
##

## First we are going to require that the secondary split
## points be to the left of L12_2 .. essentially two segment
## lengths from the right perimeter, where "segment length" is 
## computed relative to the density of the final escort group.

HB = And(HA, S01_1 < L12_2,S23_1 < L12_2)

ok = prove(Implies(HB,C0))
assert not ok, "This still isn't enough"

##
## (One Possible) Density Impossibility Theorem
##

##
## In the first case we consider situations where
## the density of the secondary escort groups are
## less than the density of the initial escort group.
##

D01_1 = EE01_1.density()
D12_0 = EE12_0.density()

HC = And(HB,D01_1 <= D12_0)
L0 = Implies(HC,C0)

ok = prove(L0)
assert ok, "We expected L0 to be True"
print("OK")

##
## (Another Possible) Reasonable Impossibility Theorem
##

## The alternative is to bound the initial split
## point to be to the right of L12_2.  This is a
## somewhat arbitrary (though sufficient) condition
## that tries to make the possible counterexamples
## "reasonable". 

ok = prove(Implies(hyps,L12_2 < S12_2))
assert ok, "L12_2 must be left of the S12_2 split point"

HD = And(HB,L12_2 <= S01_1)
L1 = Implies(HD,C0)

ok = prove(L1)
assert ok, "We expected L1 to be True"
print("OK")

OK
OK
